# Time-domain ODMR measurement (with Lock-in Amplifier & NI DAQ)

![frequency-shceduling](./figure/time-scheduling.jpg)


Import necessary modules and functions firstly

In [ ]:
import matplotlib.pyplot as plt
from odmactor.scheduler import RamseyScheduler, RabiScheduler, RelaxationScheduler
from odmactor.scheduler import HahnEchoScheduler, HighDecouplingScheduler
from odmactor.utils.plotting import plot_ramsey, plot_rabi, plot_t1
import scipy.constants as C
import numpy as np


## Ramsey detecting

**1. Set measurement parameters**
- number of readout operation $N$ per time interval point
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- calibrated MW $\pi$ pulse parameters, i.e., frequency (unit: Hz), power (unit: dBm) and time (unit: s)
- laser initialization time $t_{init}$, signal readout pulse time $t_{sig}$, etc.
- optional parameters: MW power $p$, if designed, the time of built-in MW $\pi$ pulse will be regulated correspondingly
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:

t_start = 20
t_end = 1500
t_step = 20

pi_power = -20
pi_freq = 44 * C.mega
pi_time = 150 * C.nano

t_init = 4e3
inter_init_mw = 1000
inter_mw_read = 1000
pre_read = 0
t_read_sig = 3000
# t_read_ref = t_read_sig

N = int(1e6)

**2. Construct a Ramsey Scheduler and run**

Especially, the calibrated MW $\pi$ pulse should be configured


In [ ]:
scheduler = RamseyScheduler(mw_ttl=1, with_ref=False, epoch_omit=10, use_lockin=True)
scheduler.configure_mw_paras(power=pi_power, freq=pi_freq)
scheduler.pi_pulse['freq'], scheduler.pi_pulse['power'], scheduler.pi_pulse['time'] = pi_freq, pi_power, pi_time
scheduler.configure_lockin_counting()
scheduler.configure_odmr_seq(t_init, t_read_sig, inter_init_mw=inter_init_mw, inter_mw_read=inter_mw_read,
                             pre_read=pre_read, N=N)
scheduler.set_delay_times(t_start, t_end, t_step)

In [ ]:
scheduler.run_scanning()
scheduler.close()

In [ ]:
# observe detection sequence
scheduler.gene_pseudo_detect_seq()
fig = scheduler.sequences_figure

**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
if len(counts_sig_ref) == 3:
    contrast_ramsey = [(sig - ref)  for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]
else:
    contrast_ramsey = []


In [ ]:
plt.plot(counts_sig_ref[0], counts_sig_ref[1], 'o-')
if len(counts_sig_ref) == 3:
    plt.plot(counts_sig_ref[0], counts_sig_ref[2], 'o-')
plt.title('Ramsey')
plt.xlabel('Duration time (ns)')
plt.ylabel('Count')


In [ ]:
if len(contrast_ramsey) == 3:
    plt.plot(counts_sig_ref[0][1:], contrast_ramsey[1:], 'o-')
    plt.title('Ramsey')

In [ ]:
np.savetxt('ramsey_44mhz_150ns_2.txt', scheduler.result)

## Rabi oscillation

**1. Set measurement parameters**
- number of readout operation $N$ per time interval point
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- laser initilization time $t_{init}$, singal readout pulse time $t_{sig}$, etc.
- MW power $p$, MW frequency $f$
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:
t_start = 20
t_end = 1500
t_step = 25

t_init = 4000
inter_init_mw = 1000
inter_mw_read = 1000

t_read_sig = 3000
pre_read = 0

N = int(1e6)

p = -20
f = 46 * C.mega

**2. Construct a Rabi measurement Scheduler and run**

In [ ]:
scheduler = RabiScheduler(mw_ttl=1, with_ref=False, epoch_omit=5, use_lockin=True)
scheduler.configure_mw_paras(power=p, freq=f)
scheduler.configure_lockin_counting()
scheduler.configure_odmr_seq(t_init, t_read_sig, pre_read=pre_read, inter_init_mw=inter_init_mw,
                             inter_mw_read=inter_mw_read, N=N)
scheduler.set_delay_times(t_start, t_end, t_step)

# scheduler.asg_dwell

In [ ]:
scheduler.run_scanning()

scheduler.close()

In [ ]:
scheduler.gene_pseudo_detect_seq()
fig = scheduler.sequences_figure

**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
if len(counts_sig_ref) == 3:
    contrast = [sig / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]

In [ ]:
# plot_rabi(counts_sig_ref[0][1:], counts_sig_ref[1][1:])
plt.style.use('seaborn')
#plt.vlines(210, 13.6,14)
plt.plot(counts_sig_ref[0][1:], counts_sig_ref[1][1:], 'o', label='signal')
if len(counts_sig_ref) == 3:
    plt.plot(counts_sig_ref[0], counts_sig_ref[2], 'o-', label='referengce')
plt.legend()
plt.title('Rabi Oscillation')
plt.xlabel('MW time (ns)')
plt.ylabel('counts')

In [ ]:
if contrast:
    plt.plot(counts_sig_ref[0][1:], contrast[1:], 'o-')

In [ ]:
np.savetxt('rabi_44.2mhz_-25dbm_sic.txt', scheduler.result)


## T1 relaxometry

**1. Set measurement parameters**
- number of readout operation $N$ per time interval point
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- laser initialization time $t_{init}$, signal readout pulse time $t_{sig}$, etc.
- calibrated MW $\pi$ pulse parameters, i.e., frequency (unit: Hz), power (unit: dBm) and time (unit: s)
- optional parameters: MW power $p$, if designed, the time of built-in MW $\pi$ pulse will be regulated correspondingly
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:

t_start = 100
t_end = 1e6
# t_step = 1e4

pi_power = -30
pi_freq = 2.852 * C.giga
pi_time = 210 * C.nano

t_init = 3e3
inter_init_mw = 1e3
t_read_sig = 800
t_read_ref = t_read_sig

N = int(1e6 / 4)

# times = np.round(10 ** np.linspace(1, 5, 50)) * 10



**Construct a T1 measurement scheduler and run**

In [ ]:
scheduler = RelaxationScheduler(mw_ttl=1, ms=0, use_lockin=True)
scheduler.pi_pulse['freq'], scheduler.pi_pulse['power'], scheduler.pi_pulse['time'] = pi_freq, pi_power, pi_time
scheduler.configure_lockin_counting()
scheduler.configure_odmr_seq(t_init, t_read_sig, inter_init_mw=inter_init_mw, N=N)
scheduler.set_delay_times(t_start, t_end, length=51, logarithm=True) # exponential step interval
# scheduler.set_delay_times(times=times)

print('scheduler for detecting relaxation time of state Ms={}'.format(scheduler.ms))

In [ ]:
scheduler.run_scanning()
scheduler.close()


In [ ]:
# observe sequence figure
scheduler.gene_pseudo_detect_seq()
fig = scheduler.sequences_figure

**3. Calculate contrast and plot figures**

In [ ]:
## count & contrast
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
if len(counts_sig_ref) == 3:
    contrast = [sig / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]

In [ ]:
plot_t1(counts_sig_ref[0], contrast)



## Hahn echo measurement

**1. Set measurement parameters**
- number of readout operation $N$ per time interval point
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- calibrated MW $\pi$ pulse parameters, i.e., frequency (unit: Hz), power (unit: dBm) and time (unit: s)
- laser initialization time $t_{init}$, signal readout pulse time $t_{sig}$, etc.
- optional parameters: MW power $p$, if designed, the time of built-in MW $\pi$ pulse will be regulated correspondingly
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:
t_start = 20
t_end = 200000
t_step = 1000

pi_power = -20
pi_freq = 44 * C.mega
pi_time = 150 * C.nano

t_init = 10e3
inter_init_mw = 1000
inter_mw_read = 1000
pre_read = 0
t_read_sig = 5000
# t_read_ref = t_read_sig

N = int(1e6 / 2)


**2. Construct a Hahn echo measurement Scheduler and run**

In [ ]:
scheduler = HahnEchoScheduler(mw_ttl=0, with_ref=False, epoch_omit=5, use_lockin=True)
scheduler.configure_mw_paras(power=pi_power, freq=pi_freq)
scheduler.pi_pulse['time'], scheduler.pi_pulse['freq'], scheduler.pi_pulse['power'] = pi_time, pi_freq, pi_power
scheduler.configure_lockin_counting()
scheduler.configure_odmr_seq(t_init, t_read_sig, inter_init_mw, inter_mw_read, pre_read, N=N)
scheduler.set_delay_times(t_start,t_end,t_step)

In [ ]:
scheduler.run_scanning()
scheduler.close()

**3. Calculate contrast and plot figures**

In [ ]:
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
if len(counts_sig_ref) == 3:
    contrast = [sig / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]


In [ ]:
# plot_rabi(counts_sig_ref[0], counts_sig_ref[1])
plt.style.use('seaborn')
#plt.vlines(210, 13.6,14)
plt.plot(counts_sig_ref[0], counts_sig_ref[1], 'o--', label='signal')
if len(counts_sig_ref) == 3:
    plt.plot(counts_sig_ref[0], counts_sig_ref[2], 'o-', label='referengce')
plt.legend()
plt.title('Hahn echo')
plt.xlabel('Free precession (ns)')
plt.ylabel('counts')

In [ ]:
if contrast:
    plt.plot(counts_sig_ref[0][1:], contrast[1:], 'o-')
    plt.title('Hahn echo')

## High-order dynamical decoupling measurement

**1. Set measurement parameters**
- number of readout operation $N$ per time interval point
- dynamical decoupling order, i.e., $\pi$ pulse number
- delay time range for scanning modes, i.e., `[t_start, t_end, t_step]`, unit: "ns"
- calibrated MW $\pi$ pulse parameters, i.e., frequency (unit: Hz), power (unit: dBm) and time (unit: s)
- laser initialization time $t_{init}$, signal readout pulse time $t_{sig}$, etc.
- optional parameters: MW power $p$, if designed, the time of built-in MW $\pi$ pulse will be regulated correspondingly
- time bin $t$ per readout operation will be calculated dynamically when running the scheduler on scanning-time mode

In [ ]:
t_start = 20
t_end = 200000
t_step = 1000

pi_power = -20
pi_freq = 44 * C.mega
pi_time = 150 * C.nano

t_init = 10e3
inter_init_mw = 1000
inter_mw_read = 1000
pre_read = 0
t_read_sig = 5000
# t_read_ref = t_read_sig

N = int(1e6 / 2)

d = 8

**2. Construct a high-order dynamical decoupling measurement Scheduler and run**

In [ ]:
scheduler = HighDecouplingScheduler(mw_ttl=0, with_ref=False, epoch_omit=5, use_lockin=True, order=d)
scheduler.configure_mw_paras(power=pi_power, freq=pi_freq)
scheduler.pi_pulse['time'], scheduler.pi_pulse['freq'], scheduler.pi_pulse['power'] = pi_time, pi_freq, pi_power
scheduler.configure_lockin_counting()
scheduler.configure_odmr_seq(t_init, t_read_sig, inter_init_mw, inter_mw_read, pre_read, N=N)
scheduler.set_delay_times(t_start,t_end,t_step)

In [ ]:
scheduler.run_scanning()
scheduler.close()

**3. Calculate results and plot figures**

In [ ]:
counts_sig_ref = scheduler.result  # [times, counts, counts_ref]
if len(counts_sig_ref) == 3:
    contrast = [sig / ref for sig, ref in zip(counts_sig_ref[1], counts_sig_ref[2])]


In [ ]:
plt.plot(counts_sig_ref[0], counts_sig_ref[1], 'o--', label='signal')
if len(counts_sig_ref) == 3:
    plt.plot(counts_sig_ref[0], counts_sig_ref[2], 'o-', label='referengce')
plt.legend()
plt.title('High-order DD')
plt.xlabel('MW time (ns)')
plt.ylabel('counts')

In [ ]:
if contrast:
    plt.plot(counts_sig_ref[0][1:], contrast[1:], 'o-')
    plt.title('Hahn echo')